# Step 4 : Model Evaluation 

MLFlow integration step 

### Change Working Directory 

In [2]:
import os 
print("Current Directory : ")
%pwd

Current Directory : 


'd:\\Data Science\\Codes\\02 MLOPs\\00 MLOPs Project\\research'

In [3]:
os.chdir("../")
print("Current Directory : ")
%pwd

Current Directory : 


'd:\\Data Science\\Codes\\02 MLOPs\\00 MLOPs Project'

In [6]:
import tensorflow as tf 
# model = tf.keras.models.load_model("artifacts/training/model.h5") 

# Data Class (Structure)
Blue print of data variables needed for this step of pipeline 

In [8]:
from dataclasses import dataclass 
from pathlib import Path 

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model : Path
    training_data : Path 
    all_params : dict 
    mlflow_uri : str 
    params_image_size : list
    params_batch_size : int 

# Configuration Manager 

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories,save_json 

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_PATH_YAML,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath),
        self.params = read_yaml(params_filepath),
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self)-> EvaluationConfig:

        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/asadullahshehbaz/MLFlow_DVC_Kidney_Disease_Classification.mflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )

        return eval_config

# Evaluation Class

In [12]:
import tensorflow as tf
from pathlib import Path 
import mlflow
import mlflow.keras
from urllib.parse import urlparse 

In [ ]:
class Evaluation:
    def __init__(self,config : EvaluationConfig):
        self.config = config 

        
